Improvement: Using sequence packing.

# Tiny Stories Hackathon
> From Cluster of stars study group

## TinyStories Hackathon Rules
This hackathon is intended to be a fun competition to give ourselves practice pretraining LLMs on consumer hardware. We will follow the [TinyStories paper](<https://arxiv.org/abs/2305.07759>) and train small language models on small datasets and hardware.

The hackathon will end on April 7th, [AOE](<https://en.wikipedia.org/wiki/AoE>).

### Datasets
1. [**TinyStories:**](<https://huggingface.co/datasets/roneneldan/TinyStories>)
   Note that the TinyStories dataset is split into two versions both in the HF dataset:
     - GPT-3.5 generated TinyStories
    - GPT-4 generated TinyStories
   The tar file appears to have the cleanest versions with the least number of duplicates.
2. **[Simple Wikipedia](<https://huggingface.co/datasets/lsb/simplewiki2023>)** (optional)
   This dataset can be used to give your model more world knowledge than from just the TinyStories dataset. But be careful that 
it doesn't cause your model to use words which a typical 3 to 4-year-olds doesn't understand. It may need to be cleaned.

### Evaluation
Models will be evaluated by LLM-as-a-judge following the methodology outlined in the TinyStories paper. More details including how to submit your model's outputs early next week.

### Model Size Limits
Participants will be slotted into one of the following categories based on their hardware:
- **Small**: Up to 30M parameters. Low-to-mid range laptop GPUs and Apple Silicon.
- **Medium**: Up to 60M parameters. Mid-range GPUs (including high-end laptop GPUs and Apple Silicon)
- **Large**: Up to 120M parameters. High-end GPUs and multi-GPU systems.

### Tokenizers
While you must train your model from scratch, you are welcome to use any pre-trained tokenizer or train your own tokenizer.

### Model Architecture
You are welcome to use any model architecture you want provided you stay within the parameter budget of your hardware by following the parameter counting rules below.

### Parameter Counting
The Parameter budget is the number of unique floating-point weights receiving gradient updates:
- Unique Weights: Count each distinct floating-point weight stored in the model once.
- Reuse Multiplier: For each weight, multiply by the number of distinct times it contributes to forward computation (e.g., due to layer-sharing, layer reuse, or non-standard head-sharing). Weight-tied embedding and decoder weights are the exception and are only counted once. MQA/GQA doesn't count as head-sharing.

### Teams
Teams are limited to a maximum of 2 members and must be formed and declared within the first week.

### Training Frameworks
You might want to take a look at the following libraries and frameworks and adopt one for pretraining:
- [Composer](<https://docs.mosaicml.com/projects/composer/en/stable/index.html>) and optionally [LLM Foundry](<https://github.com/mosaicml/llm-foundry>)
- [PyTorch Lightning](<https://lightning.ai/docs/pytorch/stable/>) and optionally [LitGPT](<https://github.com/Lightning-AI/litgpt>)
- Hugging Face [Trainer](<https://huggingface.co/docs/transformers/en/main_classes/trainer>), [Accelerate](<https://huggingface.co/docs/accelerate/en/index>), and optionally [Axolotl](<https://axolotl-ai-cloud.github.io/axolotl/>) (a wrapper on top of HF)
- [fastai](<https://docs.fast.ai/>) with either [fastxtend](<https://fastxtend.benjaminwarner.dev/text.huggingface.html>)/[blurr](<https://ohmeow.github.io/blurr/>)

## Data

### Dataset 

In [ ]:
from datasets import load_dataset
import tiktoken
import torch

from minai import *

## Sequence packing

Going through https://huggingface.co/blog/sirluk/llm-sequence-packing.

In [ ]:
# Setup
import torch; torch.set_printoptions(linewidth=200)
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("gpt2")
config = AutoConfig.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_config(config)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
sentence1 = "The cat sat on the mat"
sentence2 = "The dog ate my homework"
sentence3 = "My aunt is a teacher"

sentences = [sentence1, sentence2, sentence3]
tokenized_sentences = tokenizer(sentences, return_attention_mask=False, add_special_tokens=False)["input_ids"]
tokenized_sentences

[[464, 3797, 3332, 319, 262, 2603],
 [464, 3290, 15063, 616, 26131],
 [3666, 25949, 318, 257, 4701]]

In [ ]:
tokenizer.eos_token_id

50256

In [ ]:
tokenized_sentences = [t for s in tokenized_sentences for t in s + [tokenizer.eos_token_id]]
tokenized_sentences

[464,
 3797,
 3332,
 319,
 262,
 2603,
 50256,
 464,
 3290,
 15063,
 616,
 26131,
 50256,
 3666,
 25949,
 318,
 257,
 4701,
 50256]

In [ ]:
tokenizer.decode(tokenized_sentences)

'The cat sat on the mat<|endoftext|>The dog ate my homework<|endoftext|>My aunt is a teacher<|endoftext|>'

In [ ]:
tokenized_sentences = torch.tensor(tokenized_sentences)
attn_mask = torch.ones(tokenized_sentences.size(0), tokenized_sentences.size(0), dtype=torch.int).tril()
attn_mask

tensor([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,

In [ ]:
T = tokenized_sentences.size(0)
T

19

In [ ]:
# get indices of all EOS tokens
eos_indices = (tokenized_sentences == tokenizer.eos_token_id).nonzero().squeeze()
eos_indices

tensor([ 6, 12, 18])

In [ ]:
eos_indices[1:]

tensor([12, 18])

In [ ]:
eos_indices[:-1]

tensor([ 6, 12])

In [ ]:
eos_indices[[0]]+1

tensor([7])

In [ ]:
# from indices, get length of each sequence
reps = torch.cat([eos_indices[[0]]+1, eos_indices[1:] - eos_indices[:-1]])
reps

tensor([7, 6, 6])

In [ ]:
torch.repeat_interleave(eos_indices, reps)

tensor([ 6,  6,  6,  6,  6,  6,  6, 12, 12, 12, 12, 12, 12, 18, 18, 18, 18, 18, 18])

In [ ]:
torch.repeat_interleave(eos_indices, reps).view(1,-1)

tensor([[ 6,  6,  6,  6,  6,  6,  6, 12, 12, 12, 12, 12, 12, 18, 18, 18, 18, 18, 18]])

In [ ]:
# repeat each eos index n times along dimension 1 (n is the number of tokens in the sequence)
repeated_idx = torch.repeat_interleave(eos_indices, reps).view(1,-1).expand(T, -1)
repeated_idx

tensor([[ 6,  6,  6,  6,  6,  6,  6, 12, 12, 12, 12, 12, 12, 18, 18, 18, 18, 18, 18],
        [ 6,  6,  6,  6,  6,  6,  6, 12, 12, 12, 12, 12, 12, 18, 18, 18, 18, 18, 18],
        [ 6,  6,  6,  6,  6,  6,  6, 12, 12, 12, 12, 12, 12, 18, 18, 18, 18, 18, 18],
        [ 6,  6,  6,  6,  6,  6,  6, 12, 12, 12, 12, 12, 12, 18, 18, 18, 18, 18, 18],
        [ 6,  6,  6,  6,  6,  6,  6, 12, 12, 12, 12, 12, 12, 18, 18, 18, 18, 18, 18],
        [ 6,  6,  6,  6,  6,  6,  6, 12, 12, 12, 12, 12, 12, 18, 18, 18, 18, 18, 18],
        [ 6,  6,  6,  6,  6,  6,  6, 12, 12, 12, 12, 12, 12, 18, 18, 18, 18, 18, 18],
        [ 6,  6,  6,  6,  6,  6,  6, 12, 12, 12, 12, 12, 12, 18, 18, 18, 18, 18, 18],
        [ 6,  6,  6,  6,  6,  6,  6, 12, 12, 12, 12, 12, 12, 18, 18, 18, 18, 18, 18],
        [ 6,  6,  6,  6,  6,  6,  6, 12, 12, 12, 12, 12, 12, 18, 18, 18, 18, 18, 18],
        [ 6,  6,  6,  6,  6,  6,  6, 12, 12, 12, 12, 12, 12, 18, 18, 18, 18, 18, 18],
        [ 6,  6,  6,  6,  6,  6,  6, 12, 12, 12, 12, 1

In [ ]:
# create tensor with all indices from 0 to T-1 repeated T times along dimesion 1
mask_indices = torch.arange(T).view(-1,1).expand(-1, T)
mask_indices

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1],
        [ 2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2],
        [ 3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3],
        [ 4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4],
        [ 5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5],
        [ 6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6],
        [ 7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7],
        [ 8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8],
        [ 9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9],
        [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
        [11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 1

In [ ]:
# create causal mask and additionally mask out all tokens from preceeding sequences
mask = torch.ones(T, T).tril().expand(-1, -1)
mask

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1

In [ ]:
(mask_indices > repeated_idx).int()

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,

In [ ]:
mask.masked_fill_(mask_indices > repeated_idx, False)

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1

In [ ]:
def get_attention_mask_for_packed_sequence(x, token_id, eos: bool = True):
    # store sequence length in variable for easier readability
    T = tokenized_sentences.size(0)
    # get indices of all EOS tokens
    eos_indices = (tokenized_sentences == tokenizer.eos_token_id).nonzero().squeeze()
    # from indices, get length of each sequence
    reps = torch.cat([eos_indices[[0]]+1, eos_indices[1:] - eos_indices[:-1]])
    # repeat each eos index n times along dimension 1 (n is the number of tokens in the sequence)
    repeated_idx = torch.repeat_interleave(eos_indices, reps).view(1,-1).expand(T, -1)
    # create tensor with all indices from 0 to T-1 repeated T times along dimesion 1
    mask_indices = torch.arange(T).view(-1,1).expand(-1, T)
    # create causal mask and additionally mask out all tokens from preceeding sequences
    mask = torch.ones(T, T, dtype=torch.bool).tril().expand(-1, -1)
    mask.masked_fill_(mask_indices > repeated_idx, False)
    return mask

get_attention_mask_for_packed_sequence(tokenized_sentences, tokenizer.eos_token_id)

tensor([[ True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False],
        [ True,  True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False],
        [ True,  True,  True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False],
        [ True,  True,  True,  True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False],
        [ True,  True,  True,  True,  True, False, False, False, False, False, False, False, False, False, False, False, False, False, False],
        [ True,  True,  True,  True,  True,  True, False, False, False, False, False, False, False, False, False, False, False, False, False],
        [ True,  True,  True,  True,  True,  True,  True, False, False, False, False, False, False, False, False, False, False, False, False],

Position embeddings

In [ ]:
pos_ids = torch.arange(T) - torch.repeat_interleave(torch.cat([torch.tensor([0]), eos_indices+1], dim=0)[:-1], reps)
pos_ids

tensor([0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5])

Doing it in Batch

In [ ]:
sentence4 = "Rome wasn't built in a day"
sentence5 = "My hovercraft is full of eels"

sentences = [sentence4, sentence5]
tokenized_sentences2 = tokenizer(sentences, return_attention_mask=False, add_special_tokens=False)["input_ids"]
tokenized_sentences2 = torch.tensor([t for s in tokenized_sentences2 for t in s + [tokenizer.eos_token_id]])

batch = torch.nn.utils.rnn.pad_sequence(
  [tokenized_sentences, tokenized_sentences2],
  batch_first=True, padding_value=tokenizer.eos_token_id
)
batch

tensor([[  464,  3797,  3332,   319,   262,  2603, 50256,   464,  3290, 15063,   616, 26131, 50256,  3666, 25949,   318,   257,  4701, 50256],
        [   49,   462,  2492,   470,  3170,   287,   257,  1110, 50256,  3666, 20599,  3323,   318,  1336,   286,   304,  1424, 50256, 50256]])

In [ ]:
tokenized_sentences.shape, tokenized_sentences2.shape

(torch.Size([19]), torch.Size([18]))

In [ ]:
torch.nn.utils.rnn.pad_sequence([torch.ones(3), torch.ones(10)], padding_value=99).shape

torch.Size([10, 2])

In [ ]:
torch.nn.utils.rnn.pad_sequence([torch.ones(3), torch.ones(10)], padding_value=99, batch_first=True).shape

torch.Size([2, 10])

In [ ]:
B, T = batch.shape
B, T

(2, 19)

In [ ]:
batch.view(-1).shape

torch.Size([38])

In [ ]:
(batch.view(-1) == tokenizer.eos_token_id)

tensor([False, False, False, False, False, False,  True, False, False, False, False, False,  True, False, False, False, False, False,  True, False, False, False, False, False, False, False, False,
         True, False, False, False, False, False, False, False, False,  True,  True])

In [ ]:
(batch.view(-1) == tokenizer.eos_token_id).nonzero(as_tuple=True)

(tensor([ 6, 12, 18, 27, 36, 37]),)

In [ ]:
(batch.view(-1) == tokenizer.eos_token_id).nonzero().flatten() + 1

tensor([ 7, 13, 19, 28, 37, 38])

In [ ]:
eos_idx = (batch.view(-1) == tokenizer.eos_token_id).nonzero(as_tuple=True)[0] + 1
eos_idx

tensor([ 7, 13, 19, 28, 37, 38])

In [ ]:
eos_idx_expanded = torch.cat(
  [eos_idx, torch.arange(0,B*T+1,T)]
).unique().sort()[0]
eos_idx_expanded

tensor([ 0,  7, 13, 19, 28, 37, 38])

In [ ]:
normalized_idx = eos_idx_expanded - (eos_idx_expanded // T) * T
normalized_idx = torch.where(normalized_idx == 0, T, normalized_idx)
normalized_idx

tensor([19,  7, 13, 19,  9, 18, 19])

In [ ]:
reps = normalized_idx[1:] - normalized_idx[:-1]
reps = torch.where(reps < 1, normalized_idx[1:], reps)
reps

tensor([7, 6, 6, 9, 9, 1])

In [ ]:
repeated_idx = torch.repeat_interleave(
  normalized_idx[1:], reps
).view(B,1,T).expand(-1,T,-1)
repeated_idx

tensor([[[ 7,  7,  7,  7,  7,  7,  7, 13, 13, 13, 13, 13, 13, 19, 19, 19, 19, 19, 19],
         [ 7,  7,  7,  7,  7,  7,  7, 13, 13, 13, 13, 13, 13, 19, 19, 19, 19, 19, 19],
         [ 7,  7,  7,  7,  7,  7,  7, 13, 13, 13, 13, 13, 13, 19, 19, 19, 19, 19, 19],
         [ 7,  7,  7,  7,  7,  7,  7, 13, 13, 13, 13, 13, 13, 19, 19, 19, 19, 19, 19],
         [ 7,  7,  7,  7,  7,  7,  7, 13, 13, 13, 13, 13, 13, 19, 19, 19, 19, 19, 19],
         [ 7,  7,  7,  7,  7,  7,  7, 13, 13, 13, 13, 13, 13, 19, 19, 19, 19, 19, 19],
         [ 7,  7,  7,  7,  7,  7,  7, 13, 13, 13, 13, 13, 13, 19, 19, 19, 19, 19, 19],
         [ 7,  7,  7,  7,  7,  7,  7, 13, 13, 13, 13, 13, 13, 19, 19, 19, 19, 19, 19],
         [ 7,  7,  7,  7,  7,  7,  7, 13, 13, 13, 13, 13, 13, 19, 19, 19, 19, 19, 19],
         [ 7,  7,  7,  7,  7,  7,  7, 13, 13, 13, 13, 13, 13, 19, 19, 19, 19, 19, 19],
         [ 7,  7,  7,  7,  7,  7,  7, 13, 13, 13, 13, 13, 13, 19, 19, 19, 19, 19, 19],
         [ 7,  7,  7,  7,  7,  7,  7, 13, 1

In [ ]:
mask_indices = torch.arange(T).view(1,-1,1).expand(B, -1, T)
# create mask
mask = torch.ones(T, T, dtype=torch.bool).tril().expand(B, -1, -1)
mask = mask.masked_fill(mask_indices >= repeated_idx, False)
mask

tensor([[[ True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False],
         [ True,  True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False],
         [ True,  True,  True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False],
         [ True,  True,  True,  True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False],
         [ True,  True,  True,  True,  True, False, False, False, False, False, False, False, False, False, False, False, False, False, False],
         [ True,  True,  True,  True,  True,  True, False, False, False, False, False, False, False, False, False, False, False, False, False],
         [ True,  True,  True,  True,  True,  True,  True, False, False, False, False, False, False, False, False, False, False, False, 

In [ ]:
mask.int()

tensor([[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
def get_attention_mask_for_packed_sequence(x, token_id, eos: bool = True):
    B, T = x.shape
    eos_idx = (x.view(-1) == token_id).nonzero(as_tuple=True)[0] + eos
    eos_idx_expanded = torch.cat([eos_idx, torch.arange(0,B*T+1,T)]).unique().sort()[0]
    normalized_idx = eos_idx_expanded - (eos_idx_expanded // T) * T
    normalized_idx = torch.where(normalized_idx == 0, T, normalized_idx)
    reps = normalized_idx[1:] - normalized_idx[:-1]
    reps = torch.where(reps < 1, normalized_idx[1:], reps)
    repeated_idx = torch.repeat_interleave(normalized_idx[1:], reps).view(B,1,T).expand(-1,T,-1)
    mask_indices = torch.arange(T).view(1,-1,1).expand(B, -1, T)
    mask = torch.ones(T, T, dtype=torch.bool).tril().expand(B, -1, -1)
    mask = mask.masked_fill(mask_indices >= repeated_idx, False)
    return mask

In [ ]:
pos_ids = (torch.arange(B*T) - torch.repeat_interleave(eos_idx_expanded[:-1], reps)).view(B,T)
pos_ids

tensor([[0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 0, 1, 2, 3, 4, 5, 6, 7, 8, 0]])

In [ ]:
get_attention_mask_for_packed_sequence(batch, tokenizer.eos_token_id).int()

tensor([[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Grab tiny stories data from hugging face.

In [ ]:
ds = load_dataset('roneneldan/TinyStories')
trn = ds['train']
val = ds['validation']
trn

Dataset({
    features: ['text'],
    num_rows: 2119719
})

In [ ]:
val

Dataset({
    features: ['text'],
    num_rows: 21990
})

For now, we can just use gpt2 tokenizer to get started.

In [ ]:
tokenizer = tiktoken.get_encoding('gpt2')

txt = trn[0]['text']
txt

'One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on her shirt.\n\nLily went to her mom and said, "Mom, I found this needle. Can you share it with me and sew my shirt?" Her mom smiled and said, "Yes, Lily, we can share the needle and fix your shirt."\n\nTogether, they shared the needle and sewed the button on Lily\'s shirt. It was not difficult for them because they were sharing and helping each other. After they finished, Lily thanked her mom for sharing the needle and fixing her shirt. They both felt happy because they had shared and worked together.'

In [ ]:
tokenizer.encode(txt)[:10]

[3198, 1110, 11, 257, 1310, 2576, 3706, 20037, 1043, 257]

In [ ]:
tokenizer.decode(tokenizer.encode(txt)[:10])

'One day, a little girl named Lily found a'

Let's encode them.

In [ ]:
def encode(b):
    b['text'] = [tokenizer.encode(o, allowed_special={'<|endoftext|>'}) for o in b['text']]
    return b

In [ ]:
ds = ds.with_transform(encode)
trn = ds['train']
val = ds['validation']
trn

Dataset({
    features: ['text'],
    num_rows: 2119719
})

Now we have numbers. We have to decode them to read text.

In [ ]:
trn[0]['text'][:5]

'One d'

In [ ]:
tokenizer.decode(trn[0]['text'])

TypeError: argument 'tokens': Can't extract `str` to `Vec`

In [ ]:
ds = load_dataset('roneneldan/TinyStories')
trn = ds['train']
val = ds['validation']
trn

Dataset({
    features: ['text'],
    num_rows: 2119719
})

In [ ]:
class TinyDataset(Dataset):
    def __init__(self, txt, tokenizer, ctx_sz):
        """Tokenize `txt` and chunk with `ctx_sz`"""
        self.inps, self.targs = [], []
        toks = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        
        for i in range(0, len(toks) - ctx_sz, ctx_sz):
            x = toks[i:i + ctx_sz]
            y = toks[i + 1:i + ctx_sz + 1]
            self.inps.append(torch.tensor(x))
            self.targs.append(torch.tensor(y))
    
    def __len__(self): return len(self.inps)
    
    def __getitem__(self, i): return self.inps[i], self.targs[i]

In [ ]:
trn[0]['text']

'One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on her shirt.\n\nLily went to her mom and said, "Mom, I found this needle. Can you share it with me and sew my shirt?" Her mom smiled and said, "Yes, Lily, we can share the needle and fix your shirt."\n\nTogether, they shared the needle and sewed the button on Lily\'s shirt. It was not difficult for them because they were sharing and helping each other. After they finished, Lily thanked her mom for sharing the needle and fixing her shirt. They both felt happy because they had shared and worked together.'

In [ ]:
ctx_sz = 128
ds['train'] = ds['train'][:211]

TypeError: unhashable type: 'slice'

In [ ]:
type(ds['train'])

dict

In [ ]:
eot_token = 50256

In [ ]:
trn_ds = TinyDataset(trn['text'], tokenizer, ctx_sz)
trn_ds[0]

TypeError: argument 'text': 'list' object cannot be converted to 'PyString'

### Chunk

Let's try to use very small subset of data to get started. Our goal is to add `eot_token` to the end of each text. Then, chop them up into `seq_len` to create each batch.

In [ ]:
seq_len = 256
chunk_sz = seq_len
eot_token = 50256
eot_tensor = torch.tensor([eot_token])

div_by = 2000

In [ ]:
data_len = trn.num_rows // div_by // 3
data_len

353

In [ ]:
seq_tensor = [torch.tensor(o) for o in trn[:data_len]['text']]
seq_tensor[0]

tensor([ 3198,  1110,    11,   257,  1310,  2576,  3706, 20037,  1043,   257,
        17598,   287,   607,  2119,    13,  1375,  2993,   340,   373,  2408,
          284,   711,   351,   340,   780,   340,   373,  7786,    13, 20037,
         2227,   284,  2648,   262, 17598,   351,   607,  1995,    11,   523,
          673,   714, 34249,   257,  4936,   319,   607, 10147,    13,   198,
          198,    43,   813,  1816,   284,   607,  1995,   290,   531,    11,
          366, 29252,    11,   314,  1043,   428, 17598,    13,  1680,   345,
         2648,   340,   351,   502,   290, 34249,   616, 10147,  1701,  2332,
         1995, 13541,   290,   531,    11,   366,  5297,    11, 20037,    11,
          356,   460,  2648,   262, 17598,   290,  4259,   534, 10147,   526,
          198,   198, 41631,    11,   484,  4888,   262, 17598,   290,   384,
        19103,   262,  4936,   319, 20037,   338, 10147,    13,   632,   373,
          407,  2408,   329,   606,   780,   484,   547,  7373, 

In [ ]:
cat = torch.cat([torch.cat([s, eot_tensor]) for s in seq_tensor])
cat[:200]

tensor([ 3198,  1110,    11,   257,  1310,  2576,  3706, 20037,  1043,   257,
        17598,   287,   607,  2119,    13,  1375,  2993,   340,   373,  2408,
          284,   711,   351,   340,   780,   340,   373,  7786,    13, 20037,
         2227,   284,  2648,   262, 17598,   351,   607,  1995,    11,   523,
          673,   714, 34249,   257,  4936,   319,   607, 10147,    13,   198,
          198,    43,   813,  1816,   284,   607,  1995,   290,   531,    11,
          366, 29252,    11,   314,  1043,   428, 17598,    13,  1680,   345,
         2648,   340,   351,   502,   290, 34249,   616, 10147,  1701,  2332,
         1995, 13541,   290,   531,    11,   366,  5297,    11, 20037,    11,
          356,   460,  2648,   262, 17598,   290,  4259,   534, 10147,   526,
          198,   198, 41631,    11,   484,  4888,   262, 17598,   290,   384,
        19103,   262,  4936,   319, 20037,   338, 10147,    13,   632,   373,
          407,  2408,   329,   606,   780,   484,   547,  7373, 

In [ ]:
cat.shape

torch.Size([71323])

Let's create batches with `seq_length`.

In [ ]:
trn_batches = []
for i in range(0, cat.shape[0]-seq_len, seq_len):
    trn_batches.append(cat[i:i+seq_len])
len(trn_batches)

278

### Dataset (!)

Let's create inputs and targets for a dataset.

In [ ]:
inps = complete_segments[:, :-1]
targs = complete_segments[:, 1:]
inps.shape, targs.shape

NameError: name 'complete_segments' is not defined

In [ ]:
inps[0][:20]

In [ ]:
targs[0][:20]

We can create a dataset now.

In [ ]:
trn_ds = Dataset(inps, targs)
trn_ds[0]

We got the training dataset. Now, we can get the validation dataset with the same approach.

In [ ]:
val_data_len = val.num_rows // div_by * 10
val_data_len

In [ ]:
val_seq_tensor = [torch.tensor(o) for o in val[:val_data_len]['text']]
val_seq_tensor[0]

In [ ]:
val_cat = torch.cat([torch.cat([s, eot_tensor]) for s in val_seq_tensor])
val_cat[:200]

In [ ]:
val_num_complete_segments = val_cat.size(0) // chunk_sz
val_num_complete_segments

In [ ]:
val_complete_segments = val_cat[:val_num_complete_segments * chunk_sz].view(-1, chunk_sz)
val_complete_segments.shape

In [ ]:
val_inps = val_complete_segments[:, :-1]
val_targs = val_complete_segments[:, 1:]
val_inps.shape, val_targs.shape

In [ ]:
val_ds = Dataset(val_inps, val_targs)
val_ds[0]

### DataLoader

We need a dataloader with the batch size.

TODO: do `drop_last=True` for training dataloader.

In [ ]:
bs = 4

trn_dl, val_dl = get_dls(trn_ds, val_ds, bs)
dls = DataLoaders(trn_dl, val_dl)
xb,yb = next(iter(trn_dl))
xb.shape,yb.shape

In [ ]:
xb[:5], yb[:5]

## Model

We make the model using transformer.

In [ ]:
import torch.nn as nn

### MultiHeadAttention

Here's the `MultiHeadAttention` with Causal attention.

In [ ]:
set_seed(42)
x = torch.randn((2, 2, 3)) # (bs, ctx_len, d_in)

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, ctx_len, n_head, qkv_bias=False):
        super().__init__()
        assert (d_out % n_head == 0), "d_out must be divisible by num_heads"
        self.n_head = n_head
        self.d_in = d_in
        self.d_out = d_out
        self.head_dim = d_out // n_head
        self.w_q = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.w_k = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.w_v = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.register_buffer("mask", torch.triu(torch.ones((ctx_len, ctx_len)), diagonal=1).bool())
    
    def forward(self, x): 
        bs, num_tokens, d_in = x.shape
        q = self.w_q(x)  # (bs, num_tokens, d_out)
        k = self.w_k(x)
        v = self.w_v(x)
        
        q = q.view(bs, num_tokens, self.n_head, self.head_dim)  # (bs, num_tokens, n_head, head_dim)
        k = k.view(bs, num_tokens, self.n_head, self.head_dim)
        v = v.view(bs, num_tokens, self.n_head, self.head_dim)
        
        q = q.transpose(1,2) # (bs, n_head, num_tokens, head_dim)
        k = k.transpose(1,2)
        v = v.transpose(1,2)
        
        attn_scr = q@k.transpose(2,3) # (bs, n_head, num_tokens, num_tokens)
        attn_scr = attn_scr.masked_fill(self.mask[:num_tokens, :num_tokens], -torch.inf)
        attn_wt = torch.softmax(attn_scr / k.shape[-1]**0.5, -1)
        
        ctx_vec = attn_wt@v  # (bs, n_head, num_tokens, head_dim)
        ctx_vec = ctx_vec.transpose(1,2).reshape(bs, num_tokens, -1) # (bs, num_tokens, d_out)
        
        # concat
        return ctx_vec

In [ ]:
mha = MultiHeadAttention(d_in=3, d_out=4, ctx_len=2, n_head=2)
mha(x), mha(x).shape  # Outputs (bs, num_tokens, d_out)

(tensor([[[-0.0806, -0.0211,  0.1296, -0.0414],
          [ 0.2287,  0.1592, -0.0296, -0.1997]],
 
         [[ 0.6081,  0.0087,  0.2989, -0.4998],
          [ 0.1026,  0.0422,  0.3105, -0.2781]]], grad_fn=<UnsafeViewBackward0>),
 torch.Size([2, 2, 4]))

### FeedForward

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, in_dim, hidden_dim, act=nn.ReLU()):
        super().__init__()
        self.l1 = nn.Linear(in_dim, hidden_dim)
        self.act = act
        self.l2 = nn.Linear(hidden_dim, in_dim)
    
    def forward(self, x):
        return self.l2(self.act(self.l1(x)))

In [ ]:
set_seed(42)
x = torch.randn(4)
ff = FeedForward(4, 4*4)
ff(x), ff(x).shape

### Transformer Block

In [ ]:
class TransformerBlock(nn.Module):
    def __init__(self, emb_dim, ctx_len, n_head, drop_out=0, ff_mult=4, qkv_bias=False):
        super().__init__()
        self.ln1 = nn.LayerNorm(emb_dim)
        self.ln2 = nn.LayerNorm(emb_dim)
        self.mha = MultiHeadAttention(emb_dim, emb_dim, ctx_len, n_head, qkv_bias=qkv_bias)
        self.do = nn.Dropout(drop_out)
        self.ff = FeedForward(emb_dim, emb_dim*ff_mult)
    
    def forward(self, x):
        skip1 = x
        x = self.ln1(x)
        x = self.mha(x)
        x = self.do(x)
        x = x + skip1
        
        skip2 = x
        x = self.ln2(x)
        x = self.ff(x)
        x = self.do(x)
        x = x + skip2
        return x

In [ ]:
set_seed(42)
x = torch.randn((2, 2, 3)) # (bs, ctx_len, d_in)
tb = TransformerBlock(emb_dim=3, ctx_len=2, n_head=1)
tb(x), tb(x).shape

### GPT model

In [ ]:
cfg = {
    'n_tb': 1,    # num transformer blocks
    'vocab_sz': 50257,
    'emb_dim': 48,
    'ctx_len': seq_len,
    'n_head': 1,
    'drop_out': 0,
    'drop_out_tb': 0,  # dropout within transformer blocks
    'ff_mult': 4,
    'qkv_bias': False,
     }

In [ ]:
class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.token_emb = nn.Embedding(cfg['vocab_sz'], cfg['emb_dim'])
        self.pos_emb = nn.Embedding(cfg['vocab_sz'], cfg['emb_dim'])
        self.do = nn.Dropout(cfg['drop_out'])
        self.tb = nn.Sequential(
            *[TransformerBlock(cfg['emb_dim'], cfg['ctx_len'], cfg['n_head'], cfg['drop_out_tb'],
                              cfg['ff_mult'], cfg['qkv_bias']) for _ in range(cfg['n_tb'])])
        self.final_ln = nn.LayerNorm(cfg['emb_dim'])
        self.final_l  = nn.Linear(cfg['emb_dim'], cfg['vocab_sz'])
    
    def forward(self, x):
        bs, seq_len = x.shape
        tok = self.token_emb(x)
        pos = self.pos_emb(torch.arange(seq_len, device=x.device))
        x = self.do(tok + pos)
        x = self.tb(x)
        x = self.final_ln(x)
        x = self.final_l(x)
        return x

In [ ]:
batch = xb[:3]
batch.shape

In [ ]:
set_seed(42)
model = GPTModel(cfg)
logits = model(batch)
logits.shape

In [ ]:
def get_total_params(model): return sum(p.numel() for p in model.parameters())
total_params = get_total_params(model)

In [ ]:
model.token_emb.weight.shape, model.final_l.weight.shape

In [ ]:
def get_total_memory(model):
    total_params = get_total_params(model)
    total_size_bytes = total_params * 4   # Assuming fp32
    # Convert to megabytes
    total_size_mb = total_size_bytes / (1024 * 1024)
    print(f"Total params: {total_params:,}")
    print(f"Total size: {total_size_mb:.2f} MB")

get_total_memory(model)

### Text generation

In [ ]:
def generate_text_simple(model, idx, max_new_tokens, context_size):
    # idx is (batch, n_tokens) array of indices in the current context
    for _ in range(max_new_tokens):
        idx_cond = idx[:, -context_size:]  # Crop current context if it exceeds the supported context size
        with torch.no_grad(): logits = model(idx_cond)         # (bs, n_tokens, vocab_sz)
        logits = logits[:, -1, :]                              # (bs, vocab_sz)
        probas = torch.softmax(logits, dim=-1)                 # (bs, vocab_sz)
        idx_next = torch.argmax(probas, dim=-1, keepdim=True)  # (bs, 1)
        idx = torch.cat((idx, idx_next), dim=1)                # (bs, n_tokens+1)
    return idx

In [ ]:
start_context = "Hello, I am"

encoded = tokenizer.encode(start_context)
print("encoded:", encoded)

encoded_tensor = torch.tensor(encoded).unsqueeze(0)
print("encoded_tensor.shape:", encoded_tensor.shape)

In [ ]:
model.eval() # disable dropout

out = generate_text_simple(
    model=model,
    idx=encoded_tensor, 
    max_new_tokens=6, 
    context_size=cfg["ctx_len"]
)

print("Output:", out)
print("Output length:", len(out[0]))

In [ ]:
decoded_text = tokenizer.decode(out.squeeze(0).tolist())
print(decoded_text)

For convenience, we create functions.

In [ ]:
def text_to_token_ids(text, tokenizer):
    encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
    encoded_tensor = torch.tensor(encoded).unsqueeze(0) # add batch dimension
    return encoded_tensor

In [ ]:
text_to_token_ids('wassup my dawg', tokenizer)

In [ ]:
def token_ids_to_text(token_ids, tokenizer):
    flat = token_ids.squeeze(0) # remove batch dimension
    return tokenizer.decode(flat.tolist())

In [ ]:
token_ids_to_text(text_to_token_ids('wassup my dawg', tokenizer), tokenizer)

In [ ]:
start_context = "Every effort moves you"
tokenizer = tiktoken.get_encoding("gpt2")

token_ids = generate_text_simple(
    model=model,
    idx=text_to_token_ids(start_context, tokenizer),
    max_new_tokens=10,
    context_size=cfg["ctx_len"]
)

token_ids_to_text(token_ids, tokenizer)

Sanity check

In [ ]:
print("Train loader:")
for x, y in trn_dl:
    print(x.shape, y.shape)

print("\nValidation loader:")
for x, y in val_dl:
    print(x.shape, y.shape)

In [ ]:
train_tokens = 0
for input_batch, target_batch in trn_dl:
    train_tokens += input_batch.numel()

val_tokens = 0
for input_batch, target_batch in val_dl:
    val_tokens += input_batch.numel()

print("Training tokens:", train_tokens)
print("Validation tokens:", val_tokens)
print("All tokens:", train_tokens + val_tokens)

In [ ]:
# def calc_loss_batch(input_batch, target_batch, model, device):
#     input_batch, target_batch = input_batch.to(device), target_batch.to(device)
#     logits = model(input_batch)
#     loss = torch.nn.functional.cross_entropy(logits.flatten(0, 1), target_batch.flatten())
#     return loss


# def calc_loss_loader(data_loader, model, device, num_batches=None):
#     total_loss = 0.
#     if len(data_loader) == 0:
#         return float("nan")
#     elif num_batches is None:
#         num_batches = len(data_loader)
#     else:
#         # Reduce the number of batches to match the total number of batches in the data loader
#         # if num_batches exceeds the number of batches in the data loader
#         num_batches = min(num_batches, len(data_loader))
#     for i, (input_batch, target_batch) in enumerate(data_loader):
#         if i < num_batches:
#             loss = calc_loss_batch(input_batch, target_batch, model, device)
#             total_loss += loss.item()
#         else:
#             break
#     return total_loss / num_batches

In [ ]:
import torch.nn.functional as F

In [ ]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Note:
# # Uncommenting the following lines will allow the code to run on Apple Silicon chips, if applicable,
# # which is approximately 2x faster than on an Apple CPU (as measured on an M3 MacBook Air).
# # However, the resulting loss values may be slightly different.

# #if torch.cuda.is_available():
# #    device = torch.device("cuda")
# #elif torch.backends.mps.is_available():
# #    device = torch.device("mps")
# #else:
# #    device = torch.device("cpu")
# #
# # print(f"Using {device} device.")


# model.to(device) # no assignment model = model.to(device) necessary for nn.Module classes


# torch.manual_seed(123) # For reproducibility due to the shuffling in the data loader

# with torch.no_grad(): # Disable gradient tracking for efficiency because we are not training, yet
#     train_loss = calc_loss_loader(trn_dl, model, device)
#     val_loss = calc_loss_loader(val_dl, model, device)

# print("Training loss:", train_loss)
# print("Validation loss:", val_loss)

## Learner

In [ ]:
from torcheval.metrics import  MulticlassAccuracy

In [ ]:
def loss_fn(pred, targ): return F.cross_entropy(pred.flatten(0, 1), targ.flatten())

In [ ]:
cfg = {
    'n_tb': 4,    # num transformer blocks
    'vocab_sz': 50257,
    'emb_dim': 96*2,
    'ctx_len': seq_len,
    'n_head': 4,
    'drop_out': 0,
    'drop_out_tb': 0,  # dropout within transformer blocks
    'ff_mult': 4,
    'qkv_bias': False,
}

In [ ]:
set_seed(42)
model = GPTModel(cfg)
start_context = "Once upon a time, there lived a bunny in a field. Her name was Lucy. Lucy loved to have feasts and parties with her bunny friends. One day, when Lucy was about to leave for a feast at a friend's house, she realized she's starting to feel sick. She was so weak she could"

token_ids = generate_text_simple(
    model=model,
    idx=text_to_token_ids(start_context, tokenizer),
    max_new_tokens=10,
    context_size=cfg["ctx_len"]
)

print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

In [ ]:
class LLMMetricsCB(MetricsCB):
    def __init__(self, *ms, **metrics):
        super().__init__(*ms, **metrics)
    
    def after_batch(self, learn):
        x,y,*_ = to_cpu(learn.batch)
        for m in self.metrics.values(): m.update(to_cpu(learn.preds.flatten(0, 1)), y.flatten())
        self.loss.update(to_cpu(learn.loss), weight=len(x))

In [ ]:
cbs = [LLMMetricsCB(accuracy=MulticlassAccuracy()), ProgressCB(), TrainCB()]
learn = Learner(model, dls, loss_func=loss_fn, cbs=cbs)
learn.summary()

In [ ]:
learn.lr_find(gamma=1.1, max_mult=2, start_lr=1e-3)

In [ ]:
set_seed(42)
model = GPTModel(cfg)
cbs = [LLMMetricsCB(accuracy=MulticlassAccuracy()), ProgressCB(), TrainCB()]
learn = Learner(model, dls, loss_func=loss_fn, cbs=cbs)
learn.fit(10, lr=0.1)

In [ ]:
token_ids = generate_text_simple(
    model=model,
    idx=text_to_token_ids(start_context, tokenizer),
    max_new_tokens=10,
    context_size=cfg["ctx_len"])

print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

In [ ]:
def train_model_simple(model, train_loader, val_loader, optimizer, device, num_epochs,
                       eval_freq, eval_iter, start_context, tokenizer):
    # Initialize lists to track losses and tokens seen
    train_losses, val_losses, track_tokens_seen = [], [], []
    tokens_seen, global_step = 0, -1

    # Main training loop
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        
        for input_batch, target_batch in train_loader:
            optimizer.zero_grad() # Reset loss gradients from previous batch iteration
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            loss.backward() # Calculate loss gradients
            optimizer.step() # Update model weights using loss gradients
            tokens_seen += input_batch.numel()
            global_step += 1

            # Optional evaluation step
            if global_step % eval_freq == 0:
                train_loss, val_loss = evaluate_model(
                    model, train_loader, val_loader, device, eval_iter)
                train_losses.append(train_loss)
                val_losses.append(val_loss)
                track_tokens_seen.append(tokens_seen)
                print(f"Ep {epoch+1} (Step {global_step:06d}): "
                      f"Train loss {train_loss:.3f}, Val loss {val_loss:.3f}")

        # Print a sample text after each epoch
        generate_and_print_sample(
            model, tokenizer, device, start_context
        )

    return train_losses, val_losses, track_tokens_seen


def evaluate_model(model, train_loader, val_loader, device, eval_iter):
    model.eval()
    with torch.no_grad():
        train_loss = calc_loss_loader(train_loader, model, device, num_batches=eval_iter)
        val_loss = calc_loss_loader(val_loader, model, device, num_batches=eval_iter)
    model.train()
    return train_loss, val_loss


def generate_and_print_sample(model, tokenizer, device, start_context):
    model.eval()
    context_size = model.pos_emb.weight.shape[0]
    encoded = text_to_token_ids(start_context, tokenizer).to(device)
    with torch.no_grad():
        token_ids = generate_text_simple(
            model=model, idx=encoded,
            max_new_tokens=50, context_size=context_size
        )
    decoded_text = token_ids_to_text(token_ids, tokenizer)
    print(decoded_text.replace("\n", " "))  # Compact print format
    model.train()

Hyperparameters: Learning rate, optimizer: Gradient clipping, batch size: 4k

Mixed precision -> weight decay needed. (bfloat16)

Distributed data parallel: Split data into 2 and use graident accumulation

Fully Sharded data parallel: shard of data into GPUs as layer goes.

CPU offload

DataLoader: Use for loop.

!!!!! Look at the data. !!!!!

Eval: next token accuracy, loss

Try GLU instead of ReLU

Tips: 

1. Try simple model.
2. Weight Tying.
3. Hyperparameter sweep
4. minbpe


Get sequencing packing to work -> iterate faster
flash attention.